数据集来源：https://tianchi.aliyun.com/dataset/dataDetail?dataId=50893    
该数据集包含支付宝的两个商业目标活动日志。

该数据集包含三张数据表  
emb_tb_2.csv: User feature dataset.  
effect_tb.csv: Click/Non-click dataset.  
seed_cand_tb.csv: Seed users and candidate users dataset.  

这里主要用到的是effect_tb.csv

数据属性：  
dt	Values from {1,2}. Indicates whether it's a first day log (“1”) or a second day log (“2”) for the target campaign.  
user_id	The unique ID of an Alipay user.  
label	Denotes whether a user clicked the campaign ads in that day dt.  
dmp_id	The unique ID of a targeting campaign.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 导入数据与数据清洗

In [2]:
# load data
data = pd.read_csv('./effect_tb.csv',header = None)
data.columns = ["dt","user_id","label","dmp_id"]

# 日志天数属性用不上，删除该列
data = data.drop(columns = "dt")
data.head(3)

,user_id,label,dmp_id
0,1,0,1
1,1000004,0,1
2,1000004,0,2


In [3]:
data.describe()

,user_id,label,dmp_id
count,2.645958e+06,2.645958e+06,2.645958e+06
mean,3.112995e+06,1.456297e-02,1.395761e+00
std,1.828262e+06,1.197952e-01,6.920480e-01
min,1.000000e+00,0.000000e+00,1.000000e+00
25%,1.526772e+06,0.000000e+00,1.000000e+00
50%,3.062184e+06,0.000000e+00,1.000000e+00
75%,4.721132e+06,0.000000e+00,2.000000e+00
max,6.265402e+06,1.000000e+00,3.000000e+00


In [4]:
# 行数与列数
data.shape

(2645958, 3)

In [5]:
# 可直接统计dataframe中每列的不同值的个数
data.nunique()

user_id    2410683
label            2
dmp_id           3
dtype: int64

In [6]:
data[data.duplicated(keep = False)].sort_values(by = ["user_id"])

,user_id,label,dmp_id
8529,1027,0,1
1485546,1027,0,1
1579415,1471,0,1
127827,1471,0,1
404862,2468,0,1
...,...,...,...
1382121,6264633,0,1
1382245,6264940,0,1
2575140,6264940,0,1
1382306,6265082,0,3


In [7]:
# drop duplicate
data = data.drop_duplicates()

# check if any duplicates left
data[data.duplicated(keep = False)]

,user_id,label,dmp_id


In [8]:
# check null values
data.info(null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2632975 entries, 0 to 2645957
Data columns (total 3 columns):
 #   Column   Non-Null Count    Dtype
---  ------   --------------    -----
 0   user_id  2632975 non-null  int64
 1   label    2632975 non-null  int64
 2   dmp_id   2632975 non-null  int64
dtypes: int64(3)
memory usage: 80.4 MB


In [9]:
data.pivot_table(index = "dmp_id", columns = "label", values = "user_id",
                aggfunc = "count", margins = True)

label,0,1,All
dmp_id,,,
1,1881745,23918,1905663
2,404811,6296,411107
3,307923,8282,316205
All,2594479,38496,2632975


In [10]:
data.dtypes

user_id    int64
label      int64
dmp_id     int64
dtype: object

# 假设检验

In [24]:
# click rate of groups
print("对  照  组： " ,data[data["dmp_id"] == 1]["label"].mean())
print("营销策略一： " ,data[data["dmp_id"] == 2]["label"].mean())
print("营销策略二： " ,data[data["dmp_id"] == 3]["label"].mean())

对  照  组：  0.012551012429794775
营销策略一：  0.015314747742072015
营销策略二：  0.026191869198779274


可以看到策略一和策略二相较对照组在点击率上都有不同程度提升。策略一提升0.2%，策略二提升1.3%。

### 样本容量检验

在进行A/B测试前，需检查样本容量是否满足试验所需最小值。  
这里借助Evan Miller的样本量计算工具：Sample Size Calculator  
我们需要设定点击率基准线以及最小提升比例（可以设置为0.2%），将对照组的点击率设为基准线

In [25]:
# sample size of campaigns
data["dmp_id"].value_counts()

1    1905663
2     411107
3     316205
Name: dmp_id, dtype: int64

两组营销活动的样本量分别为41.11万和31.62万，满足最小样本量需求。

### 策略一

检验策略一点击率的提升是否显著。

a. 零假设和备择假设  
记对照组点击率为p0，策略一点击率为p1，则：  
H0： p0 ≥ p2  
H1： p0 ＜ p1  

b. 分布类型、检验类型和显著性水平  
样本服从二点分布，独立双样本，样本大小n＞30，总体均值和标准差未知，所以采用Z检验。显著性水平α取0.05。

In [29]:
# 用户数
n_0 = len(data[data.dmp_id == 1])  # 对照组
n_1 = len(data[data.dmp_id == 2])  # 策略一

# 点击数
c_0 = len(data[data.dmp_id ==1][data.label == 1])
c_1 = len(data[data.dmp_id ==2][data.label == 1])

# 计算点击率
r_0 = c_0 / n_0
r_1 = c_1 / n_1

# 总和点击率
r = (c_0 + c_1) / (n_0 + n_1)

print("总和点击率：", r)

总和点击率： 0.013041432684297536


<ipython-input-29-b4a98501543e>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c_0 = len(data[data.dmp_id ==1][data.label == 1])
<ipython-input-29-b4a98501543e>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c_1 = len(data[data.dmp_id ==2][data.label == 1])


In [30]:
# 计算检验统计量z
z = (r_0 - r_1) / np.sqrt(r * (1 - r)*(1/n_0 + 1/n_1))

print("检验统计量z：", z)

检验统计量z： -14.165873564308429


In [31]:
# 查α=0.05对应的z分位数
from scipy.stats import norm
z_alpha = norm.ppf(0.05)
z_alpha

-1.6448536269514729

z_alpha = -1.64， 检验统计量z = -14.16。该检验为左侧单尾检验，拒绝域为{z＜z_alpha}    
所以我们可以得出结论：原假设不成立，策略一点击率的升在统计上是显著的。

### 策略二

检验策略二点击率的提升是否显著。

a. 零假设和备择假设  
记对照组点击率为p0，策略二点击率为p2，则：  
零假设 H0： p0 ≥ p2  
备择假设 H1： p0 ＜ p2  

b. 分布类型、检验类型和显著性水平  
样本服从二点分布，独立双样本，样本大小n＞30，总体均值和标准差未知，所以采用Z检验。显著性水平α取0.05。

In [33]:
# 用户数
n_2 = len(data[data.dmp_id == 3])  # 策略二

# 点击数
c_2 = len(data[data.dmp_id ==3][data.label == 1])

# 计算点击率
r_2 = c_2 / n_2

# 总和点击率
r = (c_0 + c_2) / (n_0 + n_2)

print("总和点击率：", r)

总和点击率： 0.014492310074225832


<ipython-input-33-67043077b6dc>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c_2 = len(data[data.dmp_id ==3][data.label == 1])


In [34]:
# 计算检验统计量z
z = (r_0 - r_2) / np.sqrt(r * (1 - r)*(1/n_0 + 1/n_2))

print("检验统计量z：", z)

检验统计量z： -59.44168632985996


z_alpha = -1.64， 检验统计量z = -59.44  
同理所以我们可以得出结论：原假设不成立，策略二点击率的提升在统计上是显著的。

虽然策略一和策略二都对广告点击率有提升。但是，策略二对广告点击率有显著提升效果，且相较于对照组点击率提升了近一倍，因而在两组营销策略中应选择第二组进行推广。

### 用Python函数实现假设检验

In [37]:
import statsmodels.stats.proportion as sp

#### 策略一

In [41]:
# Alternative：str in [‘two-sided’, ‘smaller’, ‘larger’]
# "替代"假设可以是双向检验，也可以是单面检验之一，smaller的意味着替代假设prop < value， 而larger的意味着prop > value。
z_score, p = sp.proportions_ztest([c_0, c_1],[n_0, n_1], alternative = "smaller")
print("检验统计量z：",z_score,"，p值：", p)

检验统计量z： -14.165873564308429 ，p值： 7.450121742737582e-46


#### 策略二

In [42]:
# 策略一检验
z_score, p = sp.proportions_ztest([c_0, c_2],[n_0, n_2], alternative = "smaller")
print("检验统计量z：",z_score,"，p值：", p)

检验统计量z： -59.44168632985996 ，p值： 0.0


# 结论
### 综上所述，两种营销策略中，策略二对广告点击率有显著提升效果，且相较于对照组点击率提升了近一倍，因而在两组营销策略中应选择第二组进行推广。